In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
langsmith_api_key = os.getenv('LANGSMITH_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_API_KEY'] = langsmith_api_key
os.environ['HUGGINGFACE_API_KEY'] = huggingface_api_key

In [2]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader # Use this
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.documents import Document

In [3]:
pdf_files = [
    r"E:\DiabetIQ\LLM\PDFs\BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf",
    r"E:\DiabetIQ\LLM\PDFs\BES-Ramadan-Guideline-2020-min.pdf",
    r"E:\DiabetIQ\LLM\PDFs\Diabetes_Care_BADAS_guideline2019-3.pdf",
    r"E:\DiabetIQ\LLM\PDFs\Insulin-Guideline-min.pdf"
]

all_docs = [] # Will store LangChain Document objects

print("Loading and Processing PDFs...")
for pdf_path in pdf_files:
    try:
        # Extract filename for metadata
        file_name = os.path.basename(pdf_path)
        print(f"-> Loading: {file_name}")

        loader = PyPDFLoader(pdf_path)
        # Load pages as individual documents. Each doc will have metadata['page']
        pages = loader.load_and_split() # This does basic splitting

        # Add source filename to metadata for each page/document
        for page_doc in pages:
            page_doc.metadata['source'] = file_name
            # Optional: clean up page content slightly if needed
            # page_doc.page_content = page_doc.page_content.replace('\n', ' ').strip()

        all_docs.extend(pages)
        print(f"   Loaded {len(pages)} pages.")

    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")

print(f"\nTotal documents loaded: {len(all_docs)}")
if all_docs:
    print("\nSample Document Metadata (first doc):")
    print(all_docs[0].metadata)
    print("\nSample Document Content (first 500 chars of first doc):")
    print(all_docs[0].page_content[:500])
else:
    print("\nNo documents were loaded successfully.")
    # Consider exiting or handling this error appropriately
    exit()

Loading and Processing PDFs...
-> Loading: BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf
   Loaded 38 pages.
-> Loading: BES-Ramadan-Guideline-2020-min.pdf
   Loaded 46 pages.
-> Loading: Diabetes_Care_BADAS_guideline2019-3.pdf
   Loaded 79 pages.
-> Loading: Insulin-Guideline-min.pdf
   Loaded 93 pages.

Total documents loaded: 256

Sample Document Metadata (first doc):
{'producer': 'Nitro PDF PrimoPDF', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': '2020-06-07T20:17:39-06:00', 'moddate': '2020-06-07T20:17:39-06:00', 'title': 'Microsoft Word - BES COVID Pract Recomnd 06 June Final Copy', 'author': 'Mir', 'source': 'BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}

Sample Document Content (first 500 chars of first doc):
Bangladesh Endocrine Society (BES) 
Practical Recommendations for Management of 
Diabetes and Other Endocrine Diseases in Patients with 
COVID-19 
 
 
 
 
 
Published Online June 2020 
 
 
All rights res

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    # Keep separators that make sense for text structure
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    length_function=len,
)

In [6]:
chunks = text_splitter.split_documents(all_docs)

print(f"\nTotal chunks created: {len(chunks)}")
if chunks:
    print("\nSample Chunk Metadata (first chunk):")
    print(chunks[0].metadata)
    print("\nSample Chunk Content (first 500 chars):")
    print(chunks[0].page_content[:500])
else:
    print("\nNo chunks were created. Check splitting process.")
    exit()


Total chunks created: 702

Sample Chunk Metadata (first chunk):
{'producer': 'Nitro PDF PrimoPDF', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': '2020-06-07T20:17:39-06:00', 'moddate': '2020-06-07T20:17:39-06:00', 'title': 'Microsoft Word - BES COVID Pract Recomnd 06 June Final Copy', 'author': 'Mir', 'source': 'BES-COVID-Pract-Recomnd-06-June-Final-Copy.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}

Sample Chunk Content (first 500 chars):
Bangladesh Endocrine Society (BES) 
Practical Recommendations for Management of 
Diabetes and Other Endocrine Diseases in Patients with 
COVID-19 
 
 
 
 
 
Published Online June 2020 
 
 
All rights reserved by: Bangladesh Endocrine Society (BES) 
 
 
Published by 
Bangladesh Endocrine Society (BES) 
Website: http://bes-org.net 
E-mail: 
endobd2012@gmail.com


In [8]:
print("\nInitializing Embedding Model...")
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/e5-small-v2")

print("\nCreating Vector Store (ChromaDB)...")
# Chroma.from_documents handles Document objects directly
# Consider adding persistence: persist_directory="./chroma_db_diabetiq"
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    # persist_directory="./chroma_db_diabetiq" # Uncomment to save DB locally
)
# If persisting: vectorstore.persist()

# To load later:
# vectorstore = Chroma(persist_directory="./chroma_db_diabetiq", embedding_function=embedding_model)

print("Vector Store Created.")


Initializing Embedding Model...

Creating Vector Store (ChromaDB)...
Vector Store Created.


In [9]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # Retrieve top 5 chunks

In [10]:
from langsmith import traceable

# Initialize Ollama with Mistral model
llm = OllamaLLM(
    model="mistral",
    temperature=0.7,  # Slightly lower temperature for more focused medical responses
    system="You are a medical assistant specialized in diabetes care."
)

@traceable()
def rag_bot(question: str) -> dict:
    # LangChain retriever will be automatically traced
    docs = retriever.invoke(question)
    docs_string = "\n\n".join(doc.page_content for doc in docs)

    instructions = f"""You are DiabetIQ, an AI assistant specialized in diabetes management.
    Use the following clinical sources to answer the patient's question accurately and safely.
    Follow these rules strictly:
    1. Provide only evidence-based medical information
    2. Never give dosage advice without "consult your doctor" disclaimer
    3. Flag any conflicting information in sources
    4. Keep answers concise (2-3 sentences maximum)
    5. If unsure, say "I recommend consulting your healthcare provider"

    Clinical Sources:
    {docs_string}

    Patient Question: {question}"""

    # Invoke Mistral via Ollama
    response = llm.invoke(instructions)
    
    return {
        "answer": response,
        "documents": docs,
        "sources": [doc.metadata.get('source', '') for doc in docs]
    }

In [12]:
from langsmith import Client

client = Client()

examples = [
    {
        "inputs": {"question": "What are common symptoms of low blood sugar (hypoglycemia)?"},
        "outputs": {"answer": "Common symptoms of hypoglycemia include shakiness, sweating, dizziness, confusion, rapid heartbeat, hunger, and irritability. Severe cases can lead to loss of consciousness. It's important to treat it promptly."},
    },
    {
        "inputs": {"question": "How can I use the DiabetIQ app to log my meals?"},
        "outputs": {"answer": "In the DiabetIQ app, navigate to the 'Log' or 'Diary' section, select 'Meal', and enter details like the food items, portion sizes, estimated carbohydrates, and the time of the meal. Saving this helps track your dietary intake."},
    },
    {
        "inputs": {"question": "Why is monitoring blood glucose levels important for diabetes management?"},
        "outputs": {"answer": "Monitoring blood glucose helps you understand how food, activity, medication, and stress affect your levels. This information empowers you and your healthcare team to make informed decisions about your treatment plan to maintain target ranges and prevent complications."},
    },
    {
        "inputs": {"question": "Can DiabetIQ help predict my risk of high blood sugar?"},
        "outputs": {"answer": "DiabetIQ uses machine learning based on your logged data (like meals, activity, glucose readings) to identify patterns and potentially indicate an increased short-term risk of high blood sugar (hyperglycemia). This feature is for informational purposes to help you be proactive and should be discussed with your healthcare provider."},
    },
    {
        "inputs": {"question": "What type of exercise is good for managing diabetes?"},
        "outputs": {"answer": "A combination of aerobic exercise (like brisk walking, swimming, cycling) and resistance training (like lifting weights or using resistance bands) is generally recommended. Always consult your doctor before starting any new exercise program to ensure it's safe and appropriate for you."}
    }
]


# Create the dataset and examples in LangSmith
dataset_name = "DiabetIQ &&A"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)

{'example_ids': ['fa2e31f9-baee-4e32-a4de-dfd9747bde6c',
  '886e39bc-50c6-4a15-afd5-bc8b70137a29',
  '65a7e0bc-522e-426e-a3da-44023d97814c',
  '9f1a9f38-365a-4960-967a-fa84937bed99',
  'e32c4fae-84fe-441c-973d-a7886a9fbf50'],
 'count': 5}

# Correctness: Response vs reference answer

In [31]:
from typing_extensions import Annotated, TypedDict

# output schema
class CorrectnessGrade(TypedDict):
    explanation: Annotated[str, ..., "Detailed clinical reasoning for diabetes management accuracy"]
    correct: Annotated[bool, ..., "True if answer is medically accurate for diabetes care"]


# prompt
correctness_instructions = """You are an expert evaluator assessing the factual accuracy of chatbot responses in DiabetIQ: an intelligent diabetes management application with an integrated LLM-augmented chatbot.

You will be given:
- A USER QUESTION (related to diabetes care, management, medications, symptoms, etc.)
- A GROUND TRUTH ANSWER (clinically accurate and verified information)
- A CHATBOT RESPONSE (generated by the DiabetIQ chatbot)

Your task is to evaluate the **correctness** of the chatbot’s response according to the following criteria:

(1) Evaluate only the factual accuracy of the chatbot response **relative to the ground truth**. Do not grade based on style, formatting, or tone.
(2) Ensure the chatbot response **does not include any incorrect or misleading information** compared to the ground truth.
(3) It is acceptable if the chatbot includes **additional medically accurate** information, as long as it does not contradict or misrepresent the ground truth.
(4) Pay special attention to **clinical accuracy**, especially for medications, insulin usage, dietary advice, blood glucose targets, or symptom interpretation.

Correctness:
- A correctness value of **True** means the chatbot's answer is entirely consistent with and accurate relative to the ground truth.
- A correctness value of **False** means the chatbot's answer contains inaccuracies, contradictions, or medically misleading content.

Provide a **step-by-step explanation** for your judgment. Do not begin by stating the correct answer. Instead, walk through your analysis comparing the chatbot response to the ground truth carefully and clearly.

Be rigorous and thoughtful in your evaluation — patient safety depends on accurate information.
"""

# LLM
grader_llm = OllamaLLM(
    model="mistral",
    temperature=0,
    system=diabetes_correctness_instructions
).with_structured_output(GroundedGrade, method="json_schema", strict=True)

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""\
QUESTION: {inputs['question']}
GROUND TRUTH ANSWER: {reference_outputs['answer']}
STUDENT ANSWER: {outputs['answer']}"""

    # Run evaluator
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions}, 
        {"role": "user", "content": answers}
    ])
    return grade["correct"]

# Relevance: Response vs input

In [32]:
# output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Clinical relevance analysis for diabetes context"]
    relevant: Annotated[bool, ..., "Whether answer properly addresses medical question"]

# Grade prompt
relevance_instructions=diabetiq_relevance_instructions = """You are an expert evaluator assessing the relevance of chatbot responses in DiabetIQ: an intelligent diabetes management application with an integrated LLM-augmented chatbot.

You will be given:
- A USER QUESTION (related to diabetes care, management, medications, lifestyle, etc.)
- A CHATBOT RESPONSE (generated by the DiabetIQ chatbot)

Your task is to evaluate the **relevance** of the chatbot’s response according to the following criteria:

(1) Ensure the CHATBOT RESPONSE is **concise** and **directly relevant** to the USER QUESTION. It should stay on topic and not introduce unrelated or tangential information.
(2) Ensure the CHATBOT RESPONSE **meaningfully helps to answer the USER QUESTION**, either by providing actionable guidance, informative clarification, or medically appropriate context.

Relevance:
- A relevance value of **True** means the chatbot's response meets both criteria and contributes directly to answering the user’s question.
- A relevance value of **False** means the chatbot's response is off-topic, overly verbose without added value, or does not clearly address the user's question.

Provide a **step-by-step explanation** for your decision. Do not begin by stating the final judgment. Instead, walk through how the response aligns (or does not align) with the user's question to ensure your reasoning is sound and clear.

This evaluation helps ensure the chatbot provides focused and helpful responses to people managing their diabetes.
"""


# LLM
relevance_llm = OllamaLLM(
    model="mistral",
    temperature=0,
    system=relevance_instructions
)



# Evaluator
def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"QUESTION: {inputs['question']}\n ANSWER: {outputs['answer']}"
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

# Groundedness: Response vs retrieved docs

In [33]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

# Grade prompt
grounded_instructions = diabetiq_groundedness_instructions = """You are an expert evaluator assessing the groundedness of chatbot responses in DiabetIQ: an intelligent diabetes management application with an integrated LLM-augmented chatbot.

You will be given:
- A set of FACTS (retrieved evidence, clinical references, or authoritative information)
- A CHATBOT RESPONSE (generated by the DiabetIQ chatbot)

Your task is to evaluate the **groundedness** of the chatbot’s response according to the following criteria:

(1) Ensure the CHATBOT RESPONSE is **based on and supported by the provided FACTS**.
(2) Ensure the CHATBOT RESPONSE does **not include hallucinated or fabricated information** that is not supported by or inferable from the FACTS.

Grounded:
- A grounded value of **True** means the chatbot's response remains strictly within the bounds of the provided FACTS and is properly supported by them.
- A grounded value of **False** means the chatbot's response includes unsupported claims, makes assumptions not backed by the facts, or introduces external/hallucinated information.

Explain your reasoning in a **step-by-step** manner to ensure your conclusion is accurate and well-justified. Focus on whether each part of the chatbot response can be traced back to the given facts.

This evaluation is crucial to ensure the chatbot provides users with reliable, evidence-based diabetes management information.
"""

# LLM 
grounded_llm =  OllamaLLM(
    model="mistral",
    temperature=0,
    system=relevance_instructions
)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([{"role": "system", "content": grounded_instructions}, {"role": "user", "content": answer}])
    return grade["grounded"]

# Retrieval relevance: Retrieved docs vs input

In [34]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Grade prompt
retrieval_relevance_instructions = """You are an expert evaluator assessing the relevance of retrieved information used by DiabetIQ: an intelligent diabetes management application with an integrated LLM-augmented chatbot.

You will be given:
- A USER QUESTION (related to diabetes care, treatment, lifestyle, or symptoms)
- A set of FACTS (documents or text segments retrieved by the system to support answering the question)

Your task is to evaluate the **retrieval relevance** of the FACTS using the following criteria:

(1) Your goal is to identify FACTS that are **completely unrelated** to the USER QUESTION.
(2) If the FACTS contain **any keywords or semantic meaning related to the QUESTION**, consider them relevant.
(3) It is acceptable if the FACTS include some unrelated information, as long as the overall content is topically or semantically related to the question.

Relevance:
- A relevance value of **True** means the FACTS contain **any** keywords, concepts, or medically relevant semantic meaning tied to the USER QUESTION and are therefore relevant.
- A relevance value of **False** means the FACTS are **entirely unrelated** to the USER QUESTION and offer no value in helping answer it.

Provide a **step-by-step explanation** of your reasoning. Begin by analyzing the QUESTION and the FACTS, then determine if any part of the retrieved content helps address the question, even partially or indirectly.

This evaluation ensures that DiabetIQ retrieves clinically meaningful information that aligns with user needs.
"""


# Grader LLM
retrieval_relevance_llm = OllamaLLM(
    model="mistral",
    temperature=0,
    system=relevance_instructions
)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nQUESTION: {inputs['question']}"

    # Run evaluator
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

# Run Evaluation

In [35]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

experiment_results = client.evaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance",
    metadata={"version": "LCEL context, gpt-4-0125-preview"},
)

experiment_results.to_pandas()

View the evaluation results for experiment: 'rag-doc-relevance-d5201a91' at:
https://smith.langchain.com/o/ff343bf1-33c1-5f3a-8dc5-4f923eff2848/datasets/1315b3c3-c3dd-46ca-bfc4-5044736dac09/compare?selectedSessions=3ec7289a-ef60-4185-8b2e-adc2abfab292




0it [00:00, ?it/s]

Error running target function: Error getting collection: Database error: error returned from database: (code: 1) no such table: collections
Traceback (most recent call last):
  File "E:\DiabetIQ\venv\Lib\site-packages\langsmith\evaluation\_runner.py", line 1915, in _forward
    fn(
    ~~^
        *args,
        ^^^^^^
        langsmith_extra=langsmith_extra,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "E:\DiabetIQ\venv\Lib\site-packages\langsmith\run_helpers.py", line 635, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_14536\3127263961.py", line 2, in target
    return rag_bot(inputs["question"])
  File "E:\DiabetIQ\venv\Lib\site-packages\langsmith\run_helpers.py", line 635, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_14536\3902628271.py", line 13, in rag_bot
    docs = retriever.invoke(ques

,inputs.question,outputs.output,error,reference.answer,feedback.wrapper,execution_time,example_id,id
0,Why is monitoring blood glucose levels importa...,None,InternalError('Error getting collection: Datab...,Monitoring blood glucose helps you understand ...,None,0.016651,65a7e0bc-522e-426e-a3da-44023d97814c,02e2ac90-f6b9-4a02-9e75-ec2b2d221c95
1,How can I use the DiabetIQ app to log my meals?,None,InternalError('Error getting collection: Datab...,"In the DiabetIQ app, navigate to the 'Log' or ...",None,0.015276,886e39bc-50c6-4a15-afd5-bc8b70137a29,da30891b-4a57-4f63-8cb8-09b5831727e5
2,Can DiabetIQ help predict my risk of high bloo...,None,InternalError('Error getting collection: Datab...,DiabetIQ uses machine learning based on your l...,None,0.014721,9f1a9f38-365a-4960-967a-fa84937bed99,3fda58a5-55a1-4f5f-ae51-bc8624e7ef30
3,What type of exercise is good for managing dia...,None,InternalError('Error getting collection: Datab...,A combination of aerobic exercise (like brisk ...,None,0.014764,e32c4fae-84fe-441c-973d-a7886a9fbf50,52534fc6-a3cd-465e-b9c9-919db2788778
4,What are common symptoms of low blood sugar (h...,None,InternalError('Error getting collection: Datab...,Common symptoms of hypoglycemia include shakin...,None,0.014338,fa2e31f9-baee-4e32-a4de-dfd9747bde6c,81c01d25-6dec-4978-a82d-d948ff8f63d1
